# Neural Network 

This notebook contains the neural network we built for classification of subjects based on EEG signals taken from the Keirn and Aunon dataset. The preprocessing and feature extraction using Burg's AR coefficients is explained in [this notebook](https://github.com/Parthiv-M/eeg-classification/blob/main/EEG%20Preprocessing%20and%20Feature%20Extraction.ipynb). The extracted features were then given as inputs to this neural network. We have tried to implement the neural network to closely resemble the original implementation as given in [this paper](https://www.cs.colostate.edu/~anderson/wp/pubs/anderson-sijercic-96.pdf). 

In [ ]:
import keras
import datetime
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from tensorflow.keras.optimizers import SGD

In [ ]:
def remove_label_columns(sub_df):
    sub_df.drop('Unnamed: 0', axis=1, inplace=True)
    mod_df = sub_df.loc[:, ~sub_df.columns.isin(['subject', 'trial'])]
    return mod_df

In [ ]:
def relace_task_with_num(sub_df):
    tasks = {
        'baseline' : 0,
        'letter-composing' : 1,
        'multiplication' : 2,
        'counting' : 3,
        'rotation' : 4
    }
    sub_df = pd.DataFrame(np.array(sub_df))
    for idx, task in enumerate(sub_df[0]):
        sub_df.at[idx, 0] = tasks[task]
    return sub_df

In [ ]:
def gen_train_data(sub_df):
    X_train = sub_df.iloc[:,1:7]
    print("X_train shape: ", X_train.shape)
    Y_train = sub_df.iloc[:,:-6]
    print("Y_train shape: ", Y_train.shape)
    X_train = np.array(X_train)
    X_train = X_train.astype(np.float)
    Y_train = np.array(Y_train)
    Y_train_cat = tf.keras.utils.to_categorical(Y_train, num_classes=5)
    Y_train_cat = Y_train_cat.astype(np.float)
    print("Y_train_cat shape: ", Y_train_cat.shape)
    return X_train, Y_train_cat

In [ ]:
def gen_val_data(val_df):
    X_val = val_df.iloc[:,1:7]
    print("X_val shape: ", X_val.shape)
    Y_val = val_df.iloc[:,:-6]
    print("Y_val shape: ", Y_val.shape)
    X_val = np.array(X_val)
    X_val = X_val.astype(np.float)
    Y_val = np.array(Y_val)
    Y_val_cat = tf.keras.utils.to_categorical(Y_val, num_classes=5)
    Y_val_cat = Y_val_cat.astype(np.float)
    print("Y_val_cat shape: ", Y_val_cat.shape)
    return X_val, Y_val_cat

In [ ]:
def gen_test_data(test_df):
    X_test = test_df.iloc[:,1:7]
    print("X_test shape: ", X_test.shape)
    Y_test = test_df.iloc[:,:-6]
    print("Y_test shape: ", Y_test.shape)
    X_test = np.array(X_test)
    X_test = X_test.astype(np.float)
    Y_test = np.array(Y_test)
    Y_test_cat = tf.keras.utils.to_categorical(Y_test, num_classes=5)
    Y_test_cat = Y_test_cat.astype(np.float)
    print("Y_test_cat shape: ", Y_test_cat.shape)
    return X_test, Y_test_cat

In [ ]:
def build_neural_network():
    model = Sequential()
    model.add(Dense(20, activation='relu', input_dim=6))
    model.add(Dense(5, activation='softmax'))
    sgd = SGD(lr=0.1, decay=0.01, momentum=0.9, nesterov=True)
    return model, sgd

In [ ]:
def build_nn_model(model, sgd):
    model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
    return model

In [ ]:
def train_model(model, X_train, Y_train, X_val, Y_val, epochs=3000, batch_size=128):
    model.fit(X_train,Y_train,epochs=epochs,batch_size=batch_size,validation_data=(X_val, Y_val))

In [ ]:
def evaluate_model(model, X_test, Y_test, batch_size=128):
    score = model.evaluate(X_test,Y_test,batch_size=batch_size)
    return score

In [ ]:
def run_subject_wise(coeff_df):
    sub_list = ['subject 1', 'subject 3', 'subject 5', 'subject 6']
    train_df_list = []
    test_df_list = []
    val_df_list = []
    score_list = []
    for subject in sub_list:
        sub_df_train = coeff_df[(coeff_df['subject'] == subject) & (coeff_df['trial'] <= 8)]
        sub_df_val = coeff_df[(coeff_df['subject'] == subject) & (coeff_df['trial'] == 9)]
        sub_df_test = coeff_df[(coeff_df['subject'] == subject) & (coeff_df['trial'] == 10)]
        
        mod_sub_df_train = remove_label_columns(sub_df_train)
        mod_sub_df_val = remove_label_columns(sub_df_val)
        mod_sub_df_test = remove_label_columns(sub_df_test)
        
        train_df_list.append(mod_sub_df_train)
        test_df_list.append(mod_sub_df_test)
        val_df_list.append(mod_sub_df_val)
        
        mod_sub_df_train = relace_task_with_num(mod_sub_df_train)
        mod_sub_df_val = relace_task_with_num(mod_sub_df_val)
        mod_sub_df_test = relace_task_with_num(mod_sub_df_test)
        
        X_train, Y_train = gen_train_data(mod_sub_df_train)
        X_val, Y_val = gen_val_data(mod_sub_df_val)
        X_test, Y_test = gen_test_data(mod_sub_df_test)
        
        model, sgd = build_neural_network()
        build_nn_model(model, sgd)
        
        start = datetime.datetime.now()
        train_model(model, X_train, Y_train, X_val, Y_val)
        end = datetime.datetime.now()
        time_diff = end - start
        print("\nTime taken for training: ", time_diff.total_seconds(), " seconds")
        
        start = datetime.datetime.now()
        score = evaluate_model(model, X_test, Y_test, batch_size=64)
        end = datetime.datetime.now()
        time_diff = end - start
        print("\nTime taken for evaluating: ", time_diff.total_seconds(), " seconds")
        
        print("\n\nScore for ", subject, " is ", score)
        
        score_list.append(score)
        
        print(subject + " done")
        print("=============================")
    return score_list

In [ ]:
def main():
    coeff_df = pd.read_csv('coefficients_1.csv')
    
    score_list = run_subject_wise(coeff_df)

    accuracies = []
    for score in score_list:
        accuracies.append(score[1])
    
    plt.plot(accuracies)

In [ ]:
if __name__ == "__main__":
    main()